In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as fits
from scipy.optimize import leastsq
from numpy import exp
from scipy import ndimage
from func import *

In [2]:
def scale_factor_spectrum(z,data,error,wo_wave):
    k = 1+z
    select1 = (wo_wave>5030*k) & (wo_wave<5050*k)
    cont_dat = data[select1]
    cont_err = error[select1]
    error_maincube = np.std(cont_dat)
    error_errcube = np.mean(cont_err)
    fact = error_maincube/error_errcube
    return fact

In [3]:
def create_1_arcsec_minicube(wo_cube,wo_err,wo_header,brightest_pixel_x,brightest_pixel_y,box_size):
    mini_cube_data = wo_cube[:,brightest_pixel_y-box_size:brightest_pixel_y+box_size+1,brightest_pixel_x-box_size:brightest_pixel_x+box_size+1]
    mini_cube_err = wo_err[:,brightest_pixel_y-box_size:brightest_pixel_y+box_size+1,brightest_pixel_x-box_size:brightest_pixel_x+box_size+1]
    wo_header['CRPIX1'] = wo_header['CRPIX1'] - (brightest_pixel_x-box_size)
    wo_header['CRPIX2'] = wo_header['CRPIX2'] - (brightest_pixel_y-box_size)
    return mini_cube_data, mini_cube_err,wo_header

In [4]:

def scaled_error(z,mini_cube,mini_err,wo_wave):
    true_err = np.zeros((np.shape(wo_wave)[0],mini_cube.shape[1],mini_cube.shape[2]),dtype=np.float32)
    for i in range(mini_cube.shape[1]):
        for j in range(mini_cube.shape[2]):
            k = 1+z
            select1 = (wo_wave>5030*k) & (wo_wave<5050*k)
   
            (spectrum,error) = (mini_cube[:,i,j],mini_err[:,i,j])  
            (spec,err) = (spectrum[select1],error[select1])
            (error_maincube,error_errcube) = (np.std(spec),np.mean(err))
            fact = error_maincube/error_errcube
            new_err = fact*error
            true_err[:,i,j] = new_err 
    return true_err
    

In [5]:
def algorithmic_script(obj,z,box_size=2,destination_path_cube="/Volumes/Seagate/HE0040-1105 Extended"):
    (wo_cube,wo_err,wo_wave,wo_header) = loadCube('%s/%s.extended_cube.fits'%(destination_path_cube,obj)) 
    [x0,y0] = [64,57]
    [brightest_pixel_x,brightest_pixel_y] = [int(x0),int(y0)]
    
    (mini_cube,mini_err,wo_header)=create_1_arcsec_minicube(wo_cube,wo_err,wo_header,brightest_pixel_x,brightest_pixel_y,box_size=7)
    true_err = scaled_error(z,mini_cube,mini_err,wo_wave)
    
#    store_cube('%s/%s/%s.3_arcsec_minicube.fits'%(destination_path_cube,obj,obj),mini_cube_Hb,mini_wave,mini_err,mini_header)
    store_cube('%s/%s.1_arcsec_minicube.fits'%(destination_path_cube,obj),mini_cube,wo_wave,true_err,wo_header)
    print ('%s'%(obj))

In [6]:
z = {"HE0040-1105":0.04196}

objs = z.keys()


for obj in objs:
    algorithmic_script(obj,z[obj])              


HE0040-1105
